In [1]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
import torch

# 确保PyTorch能够检测到GPU

In [ ]:

import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [ ]:

# Load pretrained LLM
QWEN2d5_7B_INST = "/data/models/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/acbd96531cda22292a3ceaa67e984955d3965282"
QWEN2d5_32B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ"
QWEN2d5_72B_INST_AWQ = "/data/models/hub/models--Qwen--Qwen2.5-72B-Instruct-AWQ/snapshots/035adeffb861e8ee98344e2c7d8f50fe585e373d"

#model_7Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
#tokenizer_7Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

model_32Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
tokenizer_32Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

#model_72Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")
#tokenizer_72Bn = AutoTokenizer.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")

In [ ]:

from transformers import TextStreamer

streamer = TextStreamer(tokenizer_32Bn, skip_prompt=True, skip_special_tokens=True)

In [ ]:

sys_prompt1 = """
    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.
"""


def message_to_response_via_LLM(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt")

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01, streamer=streamer)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:

chat_path = "/data/liangyunfei/data/优秀照护师与患者对话内容(未分割版).xlsx"
raw_excel = pd.read_excel(chat_path)
raw_excel


In [ ]:
raw_excel

In [ ]:
date = [i[:10] for i in raw_excel['sentAt']]

In [ ]:
date

In [47]:
#chat_path = "/data/liangyunfei/google_download/有分布要求的对话导出.csv"
#raw_excel = pd.read_csv(chat_path)

In [ ]:
chat_path = "/data/liangyunfei/data/优秀照护师与患者对话内容(未分割版).xlsx"
raw_excel = pd.read_excel(chat_path)
user_list = list(set(raw_excel['chatRoomId']))
raw_excel['sentAt'] = [i[:10] for i in raw_excel['sentAt']]
def get_QA_text(text_list, user_list):
    QA_list = []
    for i in range(len(text_list)):
        if str(text_list[i]) != '':
            if user_list[i] == '患者':
                QA_list.append('患者：' + str(text_list[i]))
            elif user_list[i] == '系统':
                QA_list.append('系统：' + str(text_list[i]))
            else:
                QA_list.append('医生：' + str(text_list[i]))
    return '\n'.join(QA_list)
raw_excel = raw_excel.fillna('')

def get_Ans_text(text_list, user_list):
    QA_list = []
    for i in range(len(text_list)):
        if str(text_list[i]) != '':
            if user_list[i] == '医生':
                QA_list.append('医生：' + str(text_list[i]))
            elif user_list[i] == '系统':
                QA_list.append('系统：' + str(text_list[i]))
    return '\n'.join(QA_list)
raw_excel = raw_excel.fillna('')




In [59]:
dateset = list(set(list(filtered_df['sentAt'])))
dateset

['2023-10-28',
 '2023-07-29',
 '2024-02-11',
 '2022-07-28',
 '2023-05-13',
 '2022-11-21',
 '2022-08-09',
 '2024-02-18',
 '2022-07-27',
 '2024-01-03',
 '2022-07-22',
 '2022-06-18',
 '2024-02-05',
 '2023-12-11',
 '2022-08-16',
 '2022-07-02',
 '2023-04-15',
 '2023-06-13',
 '2023-06-21',
 '2023-11-20',
 '2023-04-20',
 '2023-11-11',
 '2023-10-19',
 '2022-09-19',
 '2023-11-04',
 '2022-09-27',
 '2023-12-12',
 '2022-09-13',
 '2022-07-25',
 '2024-03-27',
 '2024-04-29',
 '2022-08-08',
 '2023-10-07',
 '2023-01-05',
 '2022-06-15',
 '2022-06-16',
 '2022-09-24',
 '2022-08-10',
 '2023-06-03',
 '2023-04-19',
 '2023-05-22',
 '2024-04-07',
 '2023-07-09',
 '2023-06-17',
 '2024-06-04',
 '2022-06-19',
 '2024-03-23',
 '2024-01-09',
 '2024-03-14',
 '2023-03-08',
 '2022-06-07',
 '2024-03-12',
 '2023-01-20',
 '2023-05-06',
 '2023-01-04',
 '2022-08-19',
 '2024-03-15',
 '2023-08-05',
 '2023-09-10',
 '2023-03-02',
 '2022-12-05',
 '2023-03-01',
 '2022-09-17',
 '2022-08-23',
 '2022-06-29',
 '2024-03-10',
 '2023-06-

In [64]:

filtered_df = raw_excel[raw_excel['chatRoomId'] == user_list[38]]

QA_text = get_QA_text(list(filtered_df['text']), list(filtered_df['senderType']))
QA_text.split('\n')

['系统：患者24小时未回复',
 '医生：看到消息您回复一下哦，咱们一起分析一下血糖高的原因，',
 '患者：是的和午饭一起嚼碎吃的',
 '医生：可以的，是应该嚼碎的',
 '患者：嗯嗯',
 '医生：早餐后到午餐前有再加餐吃什么东西吗？']

In [83]:
filtered_df = raw_excel[raw_excel['chatRoomId'] == user_list[49]]
dateset = list(set(list(filtered_df['sentAt'])))
for daytime in dateset:
    filtered_twice = filtered_df[filtered_df['sentAt'] == daytime]
    Ans_text = get_Ans_text(list(filtered_twice['text']), list(filtered_twice['senderType']))
    if len(Ans_text.split('\n')) > 0:
        print(daytime)
        print(Ans_text.split('\n'))

2022-07-13
['系统：官天翔您好，本周还未收到您的血压测量数值，坚持测量有助于医生了解您的血压情况并给您提供针对性的指导，请规律进行血压测量。', '系统：您之前血糖监测的习惯非常好，这周血糖控制的怎么样呢？还没收到您的血糖数据，别忘了监测哟！期待您的血糖数据，祝您健康！']
2019-11-01
['']
2020-10-27
['']
2021-07-28
['系统：官天翔您好，本周还未收到您的血压测量数值，坚持测量有助于医生了解您的血压情况并给您提供针对性的指导，请规律进行血压测量。']
2019-02-15
['']
2021-08-06
['']
2020-01-22
['系统：您好，共同照护“护血糖APP”将于除夕（1月24日）暂停线上“问医生”服务。于初四（1月28日）开始执行假日服务时间:8:00-10:00，18:00-20:00。初七（1月31日）恢复正常工作时间~']
2023-04-03
['']
2018-08-17
['系统：早餐前 7.7']
2022-01-04
['系统：您好，我们已经看到您的血糖数据，但血糖监测的频次还不足，如果能够餐前餐后配对监测，按照大夫跟您沟通的测量模组就会更好了，这样也会更好的了解您家庭血糖的波动规律。']
2023-05-13
['']
2021-06-30
['系统：官天翔您好，本周还未收到您的血压测量数值，坚持测量有助于医生了解您的血压情况并给您提供针对性的指导，请规律进行血压测量。']
2022-11-21
['']
2019-04-24
['']
2022-06-13
['']
2022-11-23
['系统：官天翔您好，本周还未收到您的血压测量数值，坚持测量有助于医生了解您的血压情况并给您提供针对性的指导，请规律进行血压测量。']
2023-10-02
['']
2024-01-03
['']
2022-01-12
['系统：官天翔您好，本周还未收到您的血压测量数值，坚持测量有助于医生了解您的血压情况并给您提供针对性的指导，请规律进行血压测量。']
2021-12-21
['']
2020-11-02
['']
2021-03-03
['']
2021-07-19
['系统：您好，我们已经看到您的血糖数据，但血糖监测的频次还不足，如果能够餐前餐后配对监测，按照大夫跟您沟通的测量模组就会

In [81]:
filtered_df = raw_excel[raw_excel['chatRoomId'] == user_list[49]]
dateset = list(set(list(filtered_df['sentAt'])))
filtered_df = filtered_df[filtered_df['sentAt'] == dateset[20]]
Ans_text = get_Ans_text(list(filtered_df['text']), list(filtered_df['senderType']))
Ans_text.split('\n')

['']

In [ ]:

len(QA_text.split('\n'))
chat_str = "\n".join(QA_text.split('\n')[:500])
len(chat_str)


In [ ]:

rule_templates = [
    "一日三餐前后的血糖波动范围应该控制在[]mmol/L以内。",
]

In [ ]:

query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "并将最终得到的规则们整理成list的形式作为输出，仅仅输出rules中的内容"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组聊天记录数据："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]

In [ ]:

llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

llm_response

#统计所有规则

In [ ]:
all_rules_list = []
for i in tqdm(range(len(user_list))):

    filtered_df = raw_excel[raw_excel['chatRoomId'] == user_list[i]]
    QA_text = get_QA_text(list(filtered_df['text']), list(filtered_df['senderType']))
    chat_str = "\n".join(QA_text.split('\n')[:500])
    res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组聊天记录数据："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]
    llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
    rules_list = llm_response.split('[')[1].split(']')[0].split(',')
    all_rules_list += rules_list

In [ ]:
all_rules_list[:100]

In [ ]:
len(''.join(all_rules_list))

In [ ]:
all_rules_list[:10]

In [ ]:
chat_str = str(all_rules_list[:550])

query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "请将它们进行语义去重，并以list的形式输出"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
rules_list1 = llm_response.split('[')[1].split(']')[0].split(',')


In [ ]:
chat_str = str(all_rules_list[:100])
char_shot = str(['\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '\n    "餐后血糖最好能控制在10mmol/L以下"',
 '\n    "患者需要定期监测餐前和餐后血糖，并上传饮食记录以便获得专业指导"',
 '\n    "患者应按医嘱用药，并注意饮食控制，必要时可适当增加餐后运动"',
 '\n    "患者若出现低血糖症状（如出汗、心慌）应及时测量血糖并采取相应措施"',
 '\n    "患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况"\n',
 '"餐前血糖应控制在7.0mmol/L以下"',
 ' "餐后血糖应控制在10.0mmol/L以下"',
 ' "餐前餐后血糖波动范围应控制在3.5mmol/L以内"',
 ' "不建议主食用水果代替"',
 ' "建议每日监测餐前餐后血糖"',
 ' "运动量不宜过大，避免引起身体不适"',
 ' "饮食中应包含足够的蔬菜和蛋白质"',
 ' "避免食用油炸食品和高糖食品"',
 ' "定期上传饮食图片以便医生进行指导"',
 '\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',])
assistant_an = str(['\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '\n    "餐后血糖最好能控制在10mmol/L以下"',
 '\n    "患者需要定期监测餐前和餐后血糖，并上传饮食记录以便获得专业指导"',
 '\n    "患者应按医嘱用药，并注意饮食控制，必要时可适当增加餐后运动"',
 '\n    "患者若出现低血糖症状（如出汗、心慌）应及时测量血糖并采取相应措施"',
 '\n    "患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况"\n',
 '"餐前血糖应控制在7.0mmol/L以下"',
 ' "不建议主食用水果代替"',
 ' "建议每日监测餐前餐后血糖"',
 ' "运动量不宜过大，避免引起身体不适"',
 ' "饮食中应包含足够的蔬菜和蛋白质"',
 ' "避免食用油炸食品和高糖食品"',
 ' "定期上传饮食图片以便医生进行指导"',
 ])
query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "请将它们进行语义去重，并以list的形式输出"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    {"role": "assistant", "content": assistant_an},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
rules_list_ = llm_response.split('[')[1].split(']')[0].split(',')

In [ ]:
len(rules_list1)

In [ ]:
# 定义一个包含文本的列表
text_list = ["这是第一行文本", "这是第二行文本", "这是第三行文本"]

# 打开一个文件用于写入（如果文件不存在会自动创建）
with open("rules.txt", "w", encoding="utf-8") as file:
    # 遍历列表中的每一个元素
    for text in all_rules_list:
        # 将每个元素写入文件，并在每个元素后添加换行符
        file.write(text + "\n")


In [ ]:
# 使用内置的 open() 函数来读取文件
file_path = 'rules.txt'  # 替换为你的记事本文件路径

with open(file_path, 'r', encoding='utf-8') as file:
    content_list = file.readlines()

In [ ]:
content_list = [ele.strip() for ele in content_list if len(ele) > 3]

In [ ]:
content_list

In [ ]:
len(content_list),content_list

In [ ]:
'    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"\n'.strip()